In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import regex as re

import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D, Conv1D
# from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

In [2]:
data = pd.read_csv("Tweets.csv")
df = (data [["textID", "text", "sentiment"]]).dropna(axis=0)

In [4]:
def preprocess_text(sen):
#     Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)


In [5]:
X = []
sentences = list(df["text"])
for sen in sentences:
    X.append(preprocess_text(sen))

In [6]:
def process (x):
    if x =="positive": return 1
    elif x=="negative": return -1
    else: return 0


X = df["text"]
y = np.array(list(map(process, df["sentiment"])))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [34]:
from gensim.models.word2vec import Word2Vec
word2vec_model = Word2Vec(X, window=3, min_count=1, workers=16)
print(word2vec_model)

token = Tokenizer(101)
token.fit_on_texts(X)
text = token.texts_to_sequences(X)
text = pad_sequences(text, 75)
print(text[:2])

X_train, X_test, y_train, y_test = train_test_split(text, y, test_size=0.20, random_state=42)


2022-09-10 20:24:09,864 : INFO : collecting all words and their counts
2022-09-10 20:24:09,865 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2022-09-10 20:24:09,866 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-09-10 20:24:09,912 : INFO : PROGRESS: at sentence #10000, processed 686846 words, keeping 99 word types
2022-09-10 20:24:09,966 : INFO : PROGRESS: at sentence #20000, processed 1368026 words, keeping 99 word types
2022-09-10 20:24:10,005 : INFO : collected 101 word types from a corpus of 1877709 raw words and 27480 sentences
2022-09-10 20:24:10,006 : INFO : Creating a fresh vocabulary
2022-09-10 20:24:10,007 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 101 unique words (100.0%% of original 101, drops 0)', 'datetime': '2022-09-10T20:24:10.007812', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit

2022-09-10 20:24:11,350 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-09-10 20:24:11,351 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-09-10 20:24:11,351 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-09-10 20:24:11,352 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-09-10 20:24:11,353 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-09-10 20:24:11,354 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-09-10 20:24:11,354 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-09-10 20:24:11,355 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-09-10 20:24:11,356 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-09-10 20:24:11,356 : INFO : EPOCH - 4 : training on 1877709 raw words (439175 effective words) took 0.4s, 1224006 effective words/s
2022-09-10 20:24:11,664 : INFO : worker thre

Word2Vec(vocab=101, vector_size=100, alpha=0.025)
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 19
  73  1 48]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 62 94
   7 91 10]]


[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 19
  73  1 48]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 62 94
   7 91 10]]


In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets
model.fit(X_train,y_train)



In [ ]:
w = (tokenizer.texts_to_sequences(["egererhe"]))
w = pad_sequences(w, padding='post', maxlen=maxlen)[0]
w
#Predict Output
predicted= model.predict([w]) # 0:Overcast, 2:Mild
print( "Predicted Value:", predicted)

In [29]:
##hyper parameters
batch_size = 32
embedding_dims = 100 #Length of the token vectors
filters = 250 #number of filters in your Convnet
kernel_size = 3 # a window size of 3 tokens
hidden_dims = 250 #number of neurons at the normal feedforward NN
epochs = 2
maxlen=100

In [36]:
model = Sequential()
# model.add(word2vec_model.wv.get_keras_embedding(True))
model.add(Conv1D(filters,kernel_size,padding = 'valid' , activation = 'relu',strides = 1 , input_shape = (maxlen,embedding_dims)))
model.add(GlobalMaxPooling1D())
#GlobalMaxPooling1D(n) default = 2.
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [37]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train,y_train,batch_size = batch_size,epochs = epochs , validation_data = (X_test,y_test))

Epoch 1/2


ValueError: in user code:

    File "C:\Users\pkris\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\pkris\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\pkris\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\pkris\anaconda3\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\pkris\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\pkris\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 o